## Archivo de creacion del mapa: FILTRO CATEGORIA PROYECTO

### 1_ Preparamos el entorno

**Importamos librerias**

* Usar KDTree para cálculos de distancia rápida:

* En lugar de usar apply para calcular la distancia, puedes utilizar scipy.spatial.KDTree para encontrar rápidamente el cliente más cercano.
  
* Optimizar la creación de marcadores

In [1]:
import re
import warnings
warnings.filterwarnings('ignore')

from math import radians
import folium
import matplotlib.cm as cm
import numpy as np
import openpyxl
import pandas as pd
from folium import CustomIcon
from folium.plugins import HeatMap, MarkerCluster
from matplotlib.colors import to_hex
from sklearn.metrics.pairwise import haversine_distances
from scipy.spatial import KDTree

# Dataframe Camiones estacionados
estacionados_camion = pd.read_excel("./estacionados_camion.xlsx", engine="openpyxl")

# Dataframe clientes 
ubi_cliente = pd.read_excel("./ubi_cliente.xlsx", engine="openpyxl")

# Tareas.xlsx
df_tareas = pd.read_excel('./Tareas-limpio.xlsx', engine="openpyxl")


In [9]:
estacionados_camion.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2288 entries, 0 to 2287
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Indice            2288 non-null   int64         
 1   Numero_de_placa   2288 non-null   object        
 2   Estado_de_viaje   2288 non-null   object        
 3   Tiempo_de_Inicio  2288 non-null   datetime64[ns]
 4   Tiempo_Final      2288 non-null   datetime64[ns]
 5   Duracion          2232 non-null   float64       
 6   Lugar_de_inicio   2288 non-null   object        
 7   camion_x          2288 non-null   float64       
 8   camion_y          2288 non-null   float64       
 9   Semana            2288 non-null   int64         
 10  Semana_del_mes    2288 non-null   int64         
dtypes: datetime64[ns](2), float64(3), int64(3), object(3)
memory usage: 196.8+ KB


In [3]:
ubi_cliente.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 587 entries, 0 to 586
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Unnamed: 0  587 non-null    int64  
 1   CODIGO      587 non-null    int64  
 2   NOMCLI      587 non-null    object 
 3   LATITUD     587 non-null    float64
 4   LONGITUD    587 non-null    float64
dtypes: float64(2), int64(2), object(1)
memory usage: 23.1+ KB


In [21]:
df_tareas.info()

<class 'pandas.core.frame.DataFrame'>
Index: 599 entries, 0 to 599
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   CODIGO    596 non-null    float64
 1   PROYECTO  599 non-null    object 
dtypes: float64(1), object(1)
memory usage: 14.0+ KB


In [6]:
# Drop null values in `CODIGO` column in `df_tareas`
df_tareas.dropna(subset = ['CODIGO'], inplace=True)

# Convert `CODIGO` from object to numeric
df_tareas['CODIGO'] = pd.to_numeric(df_tareas['CODIGO'], errors='coerce')

df_tareas_filtrado = df_tareas[['CODIGO', 'PROYECTO']]

`df_tareas_filtrado`  +  `ubi_cliente` columna `CODIGO`

In [5]:
# `df_tareas_filtrado`  `ubi_cliente` columna `CODIGO`
merged_df = pd.merge(df_tareas_filtrado, ubi_cliente, on='CODIGO', how='inner')

merged_df.drop(columns=['Unnamed: 0'], inplace=True)

# Muestra
merged_df.sample(5)

,CODIGO,PROYECTO,Unnamed: 0,NOMCLI,LATITUD,LONGITUD
112,11125.0,🗞️ Visitas periodicas Martin,2,ALEJANDRA PIRIZ,-34.750041,-55.726749
321,82019.0,Ruta Hugo Lunes,525,Kiosco (estacion atlantida. gariboti),-34.740466,-55.764763
91,85011.0,🗞️ Visitas periodicas Martin,569,KIOSCO 29,-34.743643,-55.766153
122,80073.0,🗞️ Visitas Periodicas Dario,409,Pañalera Sueños,-34.682725,-55.701305
245,22344.0,Ruta Dario Viernes,219,ANA MARTINEZ,-34.768866,-55.591854


### Cuantas veces visito un camion a un cliente

* CODIGO: El código del cliente.
* NOMCLI: El nombre del cliente.
* Numero_de_placa: La placa del camión que se estacionó cerca.
* count: Cuántas veces ese camión se estacionó cerca de ese cliente.

In [10]:
# Define a function to compute the Haversine distance
def haversine_distance(lat1, lon1, lat2, lon2):
  """
  Calcula la distancia Haversine entre dos puntos en la superficie de la Tierra.

  Args:
      lat1 (float): Latitud del primer punto en grados.
      lon1 (float): Longitud del primer punto en grados.
      lat2 (float): Latitud del segundo punto en grados.
      lon2 (float): Longitud del segundo punto en grados.

  Returns:
      float: La distancia Haversine entre los dos puntos en kilómetros.
  """
  # Radio de la Tierra en kilómetros
  R = 6371.0

  # Convertir latitud y longitud de grados a radianes
  lat1_rad = np.radians(lat1)
  lon1_rad = np.radians(lon1)
  lat2_rad = np.radians(lat2)
  lon2_rad = np.radians(lon2)

  # Diferencias en latitud y longitud
  dlat = lat2_rad - lat1_rad
  dlon = lon2_rad - lon1_rad

  # Fórmula de Haversine
  a = np.sin(dlat / 2)**2 + np.cos(lat1_rad) * np.cos(lat2_rad) * np.sin(dlon / 2)**2
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

  # Distancia   

  distance = R * c
  return distance

In [11]:
# Convert `Tiempo_de_Inicio` and `Tiempo_Final` to datetime
estacionados_camion['Tiempo_de_Inicio'] = pd.to_datetime(estacionados_camion['Tiempo_de_Inicio'])
estacionados_camion['Tiempo_Final'] = pd.to_datetime(estacionados_camion['Tiempo_Final'])

# Filter the dataframe to keep only the rows where `Estado_de_viaje` is 'Estacionamiento'
estacionados_camion_filtered = estacionados_camion[estacionados_camion['Estado_de_viaje'] == 'Estacionamiento'].copy()

# Create a cross join between `merged_df` and `estacionados_camion_filtered`
merged_df['key'] = 0
estacionados_camion_filtered['key'] = 0
cross_joined_df = pd.merge(merged_df, estacionados_camion_filtered, on='key', how='outer').drop('key', axis=1)

# Calculate the Haversine distance between each client and each parking location
cross_joined_df['distance'] = cross_joined_df.apply(lambda row: haversine_distance(row['LATITUD'], row['LONGITUD'],
                                                                                   row['camion_x'], row['camion_y']), axis=1)

# Filter the dataframe to keep only the rows where the distance is less than a threshold (e.g., 0.1)
threshold = 0.1  # Adjust this threshold as needed
filtered_df = cross_joined_df[cross_joined_df['distance'] < threshold]

# Group the filtered dataframe by `CODIGO`, `NOMCLI`, `Numero_de_placa` and count the number of times a truck parked near a client
client_truck_counts = filtered_df.groupby(['CODIGO', 'NOMCLI', 'Numero_de_placa']).size().reset_index(name='count')

# Sort the dataframe in descending order of the count
client_truck_counts = client_truck_counts.sort_values(by='count', ascending=False)


| CODIGO   | NOMCLI              | Numero_de_placa   | count   |
|:---------|:--------------------|:------------------|:--------|
| 22644    | IVAN JUAREZ         | CAA1076           | 27      |
| 22163    | CARLOS TOBLER       | CAA1076           | 27      |
| 62613    | 62613, FRINDOR      | AAW4251           | 26      |
| 81203    | 81203,BAR ATLANTIDA | AAW4251           | 25      |
| 22144    | PANADERIA FLORESTA  | CAA1076           | 23      |


In [ ]:
client_truck_counts.info()

Le metemos latitud y longitud de la ubicacion de los clientes

In [13]:
# Unir `client_truck_counts` con `merged_df` para obtener el `PROYECTO`, `LATITUD`, and `LONGITUD`
client_truck_counts_with_project = pd.merge(client_truck_counts, merged_df[['CODIGO', 'PROYECTO', 'LATITUD', 'LONGITUD']], on='CODIGO', how='left')

| CODIGO   | NOMCLI              | Numero_de_placa   | count   | PROYECTO            | LATITUD   | LONGITUD   |
|:---------|:--------------------|:------------------|:--------|:--------------------|:----------|:-----------|
| 22644    | IVAN JUAREZ         | CAA1076           | 27      | Ruta Dario Martes   | -34.7544  | -55.6757   |
| 22163    | CARLOS TOBLER       | CAA1076           | 27      | Ruta Dario Martes   | -34.7544  | -55.676    |
| 62613    | 62613, FRINDOR      | AAW4251           | 26      | Ruta Hugo Sabado    | -34.7713  | -55.7617   |
| 81203    | 81203,BAR ATLANTIDA | AAW4251           | 25      | Ruta Hugo Miercoles | -34.7717  | -55.7626   |
| 22144    | PANADERIA FLORESTA  | CAA1076           | 23      | Ruta Dario Martes   | -34.7544  | -55.6749   |


In [16]:
client_truck_counts_with_project.sort_values(by='count', ascending=False).head(20)

,CODIGO,NOMCLI,Numero_de_placa,count,PROYECTO,LATITUD,LONGITUD
0,22644.0,IVAN JUAREZ,CAA1076,27,Ruta Dario Martes,-34.754384,-55.675690
1,22163.0,CARLOS TOBLER,CAA1076,27,Ruta Dario Martes,-34.754417,-55.676015
2,62613.0,"62613, FRINDOR",AAW4251,26,Ruta Hugo Sabado,-34.771320,-55.761740
3,81203.0,"81203,BAR ATLANTIDA",AAW4251,25,Ruta Hugo Miercoles,-34.771679,-55.762630
4,22144.0,PANADERIA FLORESTA,CAA1076,23,Ruta Dario Martes,-34.754395,-55.674884
5,12951.0,"12951,ALMACEN JUAN",BYD1006,22,Ruta Martin Lunes,-34.766390,-55.826100
6,62935.0,"62935, KIOSCO DISCO",AAW4251,22,Ruta Martin Miercoles,-34.771620,-55.761350
7,11975.0,"11975, DUNAS,PARQUE DEL PLATA NORTE",CAA1076,21,Ruta Dario Lunes,-34.746430,-55.711940
8,62867.0,"62867, GORDIOLA Ma SUSANA",BYD1006,21,Ruta Martin Lunes,-34.774860,-55.762350
9,61833.0,DOBENCO S.A.,CAA1076,20,Ruta Dario Martes,-34.683840,-55.702504


In [18]:
# Abitab y red pagos
lugares_pago = {
    'CODIGO': [200000, 200002, 200003, 200005, 200006, 200007, 200008, 200009, 200010, 200011, 200012, 200013, 200014, 200015, 200016, 200017, 200018, 200019, 200020, 200021, 200022, 200023, 200024, 200025, 200026, 200027, 200028, 200029, 200030, 200031],
    'NOMCLI': ['Abitab', 'ABITAB 12/18', 'Abitab', 'Abitab Ruta 87', 'Abitab 12-21', 'Abitab', 'Abitab', 'Cambilex', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab', 'Abitab Agencia 18/09', 'Redpagos Minimercado Nico', 'Red Pagos El Dorado, Las Toscas', 'Redpagos Bachino', 'Redpagos San Luis', 'Redpagos La Tuna', 'Redpagos Provision San Jorge', 'Red Pagos', 'RedPagos', 'RedPagos', 'Cambilex', 'Redpagos Salinas', 'Redpagos Salinas Norte', 'Redpagos Pinamar', 'Redpagos Agencias Migues', 'Redpagos Saucedo'],
    'LATITUD': [-34.7773749, -34.778762, -34.7769439, -34.7722784, -34.7717067, -34.7576624, -34.7714756, -34.772715, -34.7554596, -34.7673311, -34.7506218, -34.758673, -34.769283, -34.7701505, -34.6003052, -34.79286, -34.7710358, -34.7592078, -34.7663154, -34.7801148, -34.6832898, -34.7499177, -34.7741445, -34.7679195, -34.7773725, -34.7758541, -34.77187, -34.778569, -34.4884583, -34.5999389],
    'LONGITUD': [-55.8391627, -55.8620459, -55.8489382, -55.8413815, -55.8195923, -55.766793, -55.7620991, -55.7637416, -55.7246934, -55.7220688, -55.7094748, -55.677871, -55.6553415, -55.5763933, -55.4663981, -55.49211, -55.7337499, -55.6766295, -55.5899085, -55.5607396, -55.7024884, -55.7083951, -55.7615713, -55.767487, -55.8391636, -55.8397048, -55.84086, -55.862882, -55.630918, -55.4665898]
}

# Crear DataFrame
lugares_pago_df = pd.DataFrame(lugares_pago)

# Añadir lugares empresa **Codigo 200.000 a 200032**
ubi_empresa = {
    'CODIGO': [100001, 100002, 100003, 100004, 100005, 100000],
    'NOMCLI': ['CASA DARIO', 'CASA HUGO', 'CASA CONRADO', 'CASA MIGUEL', 'CASA MARTIN', 'EMPRESA'],
    'LATITUD': [-34.770962, -34.76416, -34.766187, -34.787831, -34.761527, -34.771148],
    'LONGITUD': [-55.742166, -55.86347, -55.780495, -55.850621, -55.74736, -55.758]
}
df_ubi_empresa = pd.DataFrame(ubi_empresa)

# Coordenadas de "EMPRESA"
coords_empresa = [-34.771148, -55.758]

# Umbral de distancia para considerar que un camión está en "EMPRESA" y asi poder excluirlos del mapa
umbral_distancia = 0.00001

# Diccionario de iconos personalizados (RUTA)
iconos_camion = {
    'AAW4251': '../num_placa_PNG/AAW4251.png',
    'BERLINGO7008': '../num_placa_PNG/BERLINGO7008.png',
    'BYD1006': '../num_placa_PNG/BYD1006.png',
    'CAA1076': '../num_placa_PNG/CAA1076.png'
}

# Normalizar las claves del diccionario de iconos
iconos_camion_normalizado = {key.replace(' ', '').upper(): value for key, value in iconos_camion.items()}


# Definir los colores personalizados para cada categoría
colores_categoria = {
    'Clientes Especiales': 'yellow',
    'Visitas Periódicas': 'green',  # Recomendado
    'Rutas': 'blue',
    'Creditos': 'red',
    'Otros': 'gray'  # Recomendado
}


# Definir los íconos Font Awesome para cada categoría
iconos_categoria = {
    'Clientes Especiales': 'fa-star',
    'Visitas Periódicas': 'fa-repeat',
    'Rutas': 'fa-map-marker',
    'Creditos': 'fa-credit-card',
    'Otros': 'fa-info-circle'
}


### Mapa filtro proyectos

In [20]:
import folium

# Filtrar las columnas necesarias
filtered_df = filtered_df[['CODIGO', 'NOMCLI', 'LATITUD', 'LONGITUD', 'PROYECTO', 'Numero_de_placa', 'camion_x', 'camion_y', 'Semana_del_mes']]

# Obtener proyectos únicos y semanas del mes únicas
projects = filtered_df['PROYECTO'].unique().tolist()
weeks_of_month = filtered_df['Semana_del_mes'].unique().tolist()

# Iterar sobre cada semana del mes
for week_of_month in weeks_of_month:
    # Filtrar los datos para la semana actual
    week_data = filtered_df[filtered_df['Semana_del_mes'] == week_of_month]

    # Calcular el centro del mapa basado en las ubicaciones de los clientes
    center_lat = week_data['LATITUD'].mean()
    center_lon = week_data['LONGITUD'].mean()

    # Crear el mapa
    map_week = folium.Map(location=[center_lat, center_lon], zoom_start=12)

    # Crear FeatureGroup para cada proyecto y para los camiones
    project_feature_groups = {project: folium.FeatureGroup(name=project) for project in projects}
    truck_feature_group = folium.FeatureGroup(name='Camiones')

    # Agregar marcadores para los lugares de pago
    for index, row in lugares_pago_df.iterrows():
        folium.Marker(
            location=[row['LATITUD'], row['LONGITUD']],
            popup=f"Lugar de Pago: {row['NOMCLI']}",
            icon=folium.Icon(color='green', icon='fa-money', prefix='fa')
        ).add_to(map_week)

    # Agregar marcadores para las ubicaciones de la empresa (excepto "EMPRESA") con color celeste e icono de camión
    for index, row in df_ubi_empresa[df_ubi_empresa['CODIGO'] != 100000].iterrows():
        folium.Marker(
            location=[row['LATITUD'], row['LONGITUD']],
            popup=f"Ubicación de la empresa: {row['NOMCLI']}",
            icon=folium.Icon(color='lightblue', icon='fa-truck', prefix='fa') 
        ).add_to(map_week)

    # Iterar sobre cada fila en los datos de la semana
    for index, row in week_data.iterrows():
        # Verificar si el camión está en la ubicación de la empresa
        if haversine_distance(row['camion_x'], row['camion_y'], coords_empresa[0], coords_empresa[1]) < umbral_distancia:
            continue  # Saltar si el camión está en la empresa

        # Agregar marcador para el cliente con color e ícono según la categoría
        folium.Marker(
            location=[row['LATITUD'], row['LONGITUD']],
            popup=f"Cliente: {row['NOMCLI']}, Proyecto: {row['PROYECTO']}",
            icon=folium.Icon(color=colores_categoria.get(row['PROYECTO'], 'gray'),
                             icon=iconos_categoria.get(row['PROYECTO'], 'fa-info-circle'), prefix='fa')
        ).add_to(project_feature_groups[row['PROYECTO']])

        # Agregar marcador para el camión con ícono personalizado si está disponible.
        # Agregar el marcador al grupo de camiones
        icon_url = iconos_camion_normalizado.get(row['Numero_de_placa'], None)
        if icon_url:
            icon = folium.features.CustomIcon(icon_url, icon_size=(30, 30))
        else:
            icon = None

        folium.Marker(
            location=[row['camion_x'], row['camion_y']],
            popup=f"Camión: {row['Numero_de_placa']}",
            icon=icon
        ).add_to(truck_feature_group)

    # Agregar los FeatureGroup al mapa
    for feature_group in project_feature_groups.values():
        feature_group.add_to(map_week)

    # Agregar el FeatureGroup de camiones al mapa
    truck_feature_group.add_to(map_week)

    # Agregar control de capas para mostrar/ocultar proyectos y camiones
    folium.LayerControl().add_to(map_week)

    # Guardar el mapa
    map_week.save(f'../mapas_html/mapa_semana_{week_of_month}_filtro_categoria.html')